In [2]:
from TraydnerAPI import TraydnerAPI
from requests.exceptions import HTTPError
from typing import Optional, Dict, Any, List


# --- Example Usage ---
if __name__ == "__main__":
    # Replace "YOUR_API_KEY" with your actual API key
    API_KEY = "891830748679422e99229f7478d950fa.innE8dfKNvJ7FMeDc8afyGBwEDnRa7_UMq_8iOdIht-UOnP5rjPcd2RUPTfzefNa"
    
    # Check if the API key is just a placeholder
    if API_KEY == "YOUR_API_KEY":
        print("Please replace 'YOUR_API_KEY' with your actual API key to run the examples.")
    else:
        try:
            client = TraydnerAPI(api_key=API_KEY)
            
            # Example 1: Get account balance
            print("--- Fetching Balance ---")
            balance_data = client.get_balance()
            print(f"Balance: {balance_data.get('balance')}\n")
            
            # Example 2: Get price for BTC
            print("--- Fetching BTC Price ---")
            btc_price = client.get_price("BTC")
            print(f"Symbol: {btc_price.get('symbol')}, Price: {btc_price.get('price')}\n")

            # Example 3: Get market status for AAPL
            print("--- Fetching AAPL Market Status ---")
            aapl_status = client.get_market_status(symbol="AAPL")
            print(f"Is AAPL market open? {aapl_status.get('isOpen')}\n")
            
            # Example 4: Get historical data for AAPL
            print("--- Fetching AAPL History ---")
            aapl_history = client.get_history(symbol="AAPL", resolution="1h", limit=5)
            print(f"Fetched {aapl_history.get('count')} candles for {aapl_history.get('symbol')}")
            # print(aapl_history.get('history')) # Uncomment to see full data
            print("\n")

            # Example 5: Execute a simulated trade (use with caution)
            # print("--- Executing Simulated Trade ---")
            # trade_response = client.execute_trade(symbol="BTC", side="buy", quantity=0.01)
            # print(f"Trade Status: {trade_response.get('status')}")
            # print(trade_response)

        except ValueError as e:
            print(f"Configuration error: {e}")
        except HTTPError as e:
            print(f"API Error: {e.response.status_code} - {e.response.text}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

--- Fetching Balance ---
Balance: {'cash': 100000.0, 'stocks': {}, 'crypto': {}, 'forex': {}}

--- Fetching BTC Price ---
Symbol: BTC, Price: 113880.09

--- Fetching AAPL Market Status ---
Is AAPL market open? False

--- Fetching AAPL History ---
Fetched 5 candles for AAPL




In [5]:
# (Place in a new cell after the API and Trader class definitions)

from TraydnerAPI import TraydnerAPI
from strategies import *  # This imports all strategies
from requests.exceptions import HTTPError
from typing import Optional, Dict, Any, List
import time
import traceback

# --- User-Defined Inputs ---
API_KEY = "891830748679422e99229f7478d950fa.innE8dfKNvJ7FMeDc8afyGBwEDnRa7_UMq_8iOdIht-UOnP5rjPcd2RUPTfzefNa"

# Unified parameter structure for all strategies

STRATEGY_PARAMS = {
    "MeanReversionTrader": {
        "symbol": "BTC",
        "resolution": "1m", # Each candle = 1 minute
        "quantity": 0.01, # Amount to trade per signal
        "mean_period": 20, # Last 20 candles for moving average
        "std_dev_multiplier": 2.0, # 2 standard deviations for bands
        "poll_interval": 60 # Check every 60 seconds
    },
    "MomentumTrader": {
        "symbol": "BTC",
        "resolution": "1h",
        "quantity": 0.01,
        "rsi_period": 12, # Lookback period for RSI (Relative Strength Index)
        "oversold_threshold": 30.0,
        "overbought_threshold": 70.0,
        "poll_interval": 60*60
    }
}
# --------------------------- #

if API_KEY == "YOUR_API_KEY":
    print("ERROR: Please update 'API_KEY' in the runner script.")
else:
    try:
        print(f"\n--- Trader Bot is now running ---")
        print("Press the 'Interrupt' button (■) in the toolbar to stop.\n")

        # 1. Initialize the API client
        api_client = TraydnerAPI(api_key=API_KEY)
        
        # 2. Initialize all traders based on STRATEGY_PARAMS
        traders = []
        
        # MeanReversionTrader
        if "MeanReversionTrader" in STRATEGY_PARAMS:
            params = STRATEGY_PARAMS["MeanReversionTrader"]
            traders.append({
                "strategy": MeanReversionTrader(
                    client=api_client,
                    symbol=params["symbol"],
                    resolution=params["resolution"],
                    mean_period=params["mean_period"],
                    std_dev_multiplier=params["std_dev_multiplier"]
                ),
                "name": "MeanReversionTrader",
                "symbol": params["symbol"],
                "quantity": params["quantity"],
                "poll_interval": params["poll_interval"],
                "last_check": 0
            })
        
        # MomentumTrader
        if "MomentumTrader" in STRATEGY_PARAMS:
            params = STRATEGY_PARAMS["MomentumTrader"]
            traders.append({
                "strategy": MomentumTrader(
                    client=api_client,
                    symbol=params["symbol"],
                    resolution=params["resolution"],
                    rsi_period=params["rsi_period"],
                    oversold_threshold=params["oversold_threshold"],
                    overbought_threshold=params["overbought_threshold"]
                ),
                "name": "MomentumTrader",
                "symbol": params["symbol"],
                "quantity": params["quantity"],
                "poll_interval": params["poll_interval"],
                "last_check": 0
            })

        # ------- Strategy Initialization Complete ------- #
        
        print(f"\nInitialized {len(traders)} trading strategies.")

        # 3. Start the indefinite loop
        while True:
            current_time = time.time()
            
            for trader_config in traders:
                # Check if enough time has passed for this trader
                if current_time - trader_config["last_check"] >= trader_config["poll_interval"]:
                    strategy = trader_config["strategy"]
                    name = trader_config["name"]
                    symbol = trader_config["symbol"]
                    quantity = trader_config["quantity"]
                    
                    print(f"\n[{time.ctime()}] Checking {name} for {symbol}...")
                    
                    try:
                        # Get the signal from the trader
                        signal = strategy.get_signal()
                        
                        # Execute the trade based on signal
                        if signal == "BUY":
                            print(f"--- Executing BUY of {quantity} {symbol} ---")
                            trade_response = api_client.execute_trade(symbol, "buy", quantity)
                            print(f"Trade Response: {trade_response}")
                            
                        elif signal == "SELL":
                            print(f"--- Executing SELL of {quantity} {symbol} ---")
                            trade_response = api_client.execute_trade(symbol, "sell", quantity)
                            print(f"Trade Response: {trade_response}")
                            
                        elif signal == "HOLD":
                            print("--- No trade executed. ---")
                        
                        # Update last check time
                        trader_config["last_check"] = current_time
                        
                    except HTTPError as http_err:
                        print(f"API Error during check: {http_err}")
                    except Exception as e:
                        print(f"An unexpected error occurred: {e}")
                        traceback.print_exc()
            
            # Sleep for a short interval before checking again
            time.sleep(5)

    except KeyboardInterrupt:
        print("\nBot stopped by user (KeyboardInterrupt).")
    except NameError as e:
        print(f"\nCRITICAL ERROR: {e}")
        print("Make sure all strategy classes are properly imported.")
    except Exception as e:
        print(f"A critical error occurred on startup: {e}")
        traceback.print_exc()


--- Trader Bot is now running ---
Press the 'Interrupt' button (■) in the toolbar to stop.

MeanReversionTrader initialized for BTC.
Strategy:   Mean Reversion (Bollinger Bands)
Symbol:     BTC
Resolution: 1m
Quantity:   20
MomentumTrader initialized for BTC.
Strategy:   Momentum (RSI)
Symbol:     BTC
Resolution: 1h
RSI Period: 12
Oversold:   < 30.0
Overbought: > 70.0

Initialized 2 trading strategies.

[Mon Oct 27 22:54:33 2025] Checking MeanReversionTrader for BTC...
Fetching 20 candles of BTC@1m history...

--- Strategy Check for BTC ---
Current Price: 113880.0900
Lower Band:    47023.6744
Mean (SMA):    86852.6610
Upper Band:    126681.6476

Generated Signal: HOLD (Price is within the bands)
--- No trade executed. ---

[Mon Oct 27 22:54:38 2025] Checking MomentumTrader for BTC...
Fetching 13 candles of BTC@1h history for RSI...

--- Strategy Check for BTC ---
Current Price: 113880.0900
Current RSI:   66.46
Oversold:      < 30.0
Overbought:    > 70.0

Generated Signal: HOLD (RSI 66